In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
import math
from scipy.optimize import basinhopping
#from scipy.optimize import dual_annealing
from math import *

# from pyOpt import Optimization
# from pyOpt import SLSQP

In [2]:
import scipy
print(scipy.__version__)

1.1.0


In [47]:
## Get Non Marketing Values for each Week which can be directly used in Objective Function
path = "D:\\Affine_Projects\\WB_Warner_Bros\\MMM\\Optimisation\\22072019\\non_marketing_week_data_5channels.xlsx"
non_marketing_cols = ['const','Calender Week','Dec','Mar','Jul','1st week','2nd week','Is Action','Is Open World','same genre','different genre',
                     'Week Number','TubularViews',"ln_POSRebate","ln_TradeDisplayPlacement"]
title = "Lego Worlds"
#title = "Scribblenauts Showdown/Scribblenauts Fiesta"
channels = ["TVSpend","Facebook","Video","Display","Search"]
non_marketing_week_values = pd.read_excel(path,sheetname = "Data")
non_marketing_week_values["const"]=1
df_title_budget_constraints = pd.read_excel(path,sheetname = "Title_Budget_Constraints")
df_channel_ranges = pd.read_excel(path,sheetname = "Channel_Ranges_Weighted_Avg")
df_channel_ranges = df_channel_ranges[df_channel_ranges['Title']==title].drop(['Title'],axis=1)
df_weekwise_spending = pd.read_excel(path,sheetname = "Holiday_Weekwise_Spending_Const")
marketing_beta = pd.read_excel(path,sheetname = "marketing_betaCoeff")
#actual_week_spending = pd.read_excel(path,sheetname = "X0_Incredibles")

In [48]:
def get_title_budget(df_title_budget_constraints,title):
    return df_title_budget_constraints[df_title_budget_constraints["Title"]==title]["Budget"].values[0]

def get_Non_Marketing_Week_Values(non_marketing_week_values,non_marketing_cols,title):
    non_marketing_week_values = non_marketing_week_values[non_marketing_week_values["Title"]==title]
    non_marketing_week_values = non_marketing_week_values[non_marketing_cols]
    non_marketing_betaCoef = pd.read_excel(path,sheetname = "Beta_Coeff")
    non_marketing_week_beta_multiply = non_marketing_betaCoef.values*non_marketing_week_values
    non_marketing_week_exp = non_marketing_week_beta_multiply.apply(np.exp)
    non_marketing_week  = non_marketing_week_exp.product(axis=1)
    non_marketing = non_marketing_week.to_frame(name = "non_marketing")
    non_marketing = pd.concat([non_marketing,non_marketing_week_values[['Week Number']]],axis=1)
    return non_marketing

def get_marketing_series(x,noOfChannel):
    l = len(x)
    TVSpend = pd.Series(x[0:l:noOfChannel])
    Facebook =pd.Series(x[1:l:noOfChannel])
    Video = pd.Series(x[2:l:noOfChannel])
    Display = pd.Series(x[3:l:noOfChannel])
    Search = pd.Series(x[4:l:noOfChannel])
    return (TVSpend,Facebook,Video,Display,Search)

def get_marketing_df(TVSpend,Facebook,Video,Display,Search):
    df_TVSpend = TVSpend.to_frame(name = "TVSpend")
    df_Facebook = Facebook.to_frame(name = "Facebook")
    df_Video = Video.to_frame(name = "Video")
    df_Display = Display.to_frame(name="Display")
    df_Search = Search.to_frame(name="Search")
    return (df_TVSpend,df_Facebook,df_Video,df_Display,df_Search)


def get_ads_marketing(df_TVSpend,df_Facebook,df_Video,df_Display,df_Search,decay_sat_par_val_dict,decay_params):
    tvspend_ads = add_single_adstock_col(df_TVSpend,"TVSpend",decay_sat_par_val_dict,decay_params["TVSpend"]["decay"],decay_params["TVSpend"]["saturation"])
    facebook_ads = add_single_adstock_col(df_Facebook,"Facebook",decay_sat_par_val_dict,decay_params["Facebook"]["decay"],decay_params["Facebook"]["saturation"])
    video_ads = add_single_adstock_col(df_Video,"Video",decay_sat_par_val_dict,decay_params["Video"]["decay"],decay_params["Video"]["saturation"])
    display_ads = add_single_adstock_col(df_Display,"Display",decay_sat_par_val_dict,decay_params["Display"]["decay"],decay_params["Display"]["saturation"])
    search_ads = add_single_adstock_col(df_Search,"Search",decay_sat_par_val_dict,decay_params["Search"]["decay"],decay_params["Search"]["saturation"])

    return (tvspend_ads[tvspend_ads.columns[0]],facebook_ads[facebook_ads.columns[0]],video_ads[video_ads.columns[0]],\
            display_ads[display_ads.columns[0]],search_ads[search_ads.columns[0]])



In [49]:
def roundoff_adstock(x):
    x = round(x,4)
    if(x==0):
        return 100
    else:
        #print("X is =>",x)
        log_10 = log10(x)
        floor_log10 = floor(log_10)
        lower_power10 = pow(10,floor_log10)
        first_digit = floor(x/lower_power10)
        final = first_digit * lower_power10
        return final


def add_single_adstock_col(df,col,decay_sat_par_val_dict,decay_param,saturation_param):
    df = df[[col]]
    decay = decay_sat_par_val_dict[decay_param]
    saturation = decay_sat_par_val_dict[saturation_param]
    df["pvals_"+col]=0
    df["adstock_"+col]=0
    for index in df.index:
        curr_val = df.loc[index,col]
        if(index==0):
            df.loc[index,"pvals_"+col] = 1- (1/exp((curr_val*saturation)/roundoff_adstock(curr_val)))
            df.loc[index,"adstock_"+col] = curr_val
        else:
            prev_pvals = df.loc[index-1,"pvals_"+col]
            df.loc[index,"pvals_"+col] = 1 - ((1-prev_pvals*exp(-1*decay))/exp((curr_val*saturation)/roundoff_adstock(curr_val)))
            df.loc[index,"adstock_"+col] = curr_val+(df.loc[index-1,"adstock_"+col]*df.loc[index,"pvals_"+col])
    df["ads_"+col+"_D"+str(decay_param)+"_S"+str(saturation_param)] = df["adstock_"+col]
    df = df.drop([col,"pvals_"+col,"adstock_"+col],axis=1)
    return df



decay_sat_par_val_dict = {
    1:0.0625,
    2:0.125,
    3:0.25,
    4:0.5,
    5:1,
    6:1.5,
    7:2,
    8:2.5,
    9:0.3,
    10:0.4,
    11:0.6,
    12:0.7,
    13:0.8,
    14:1.2,
    15:1.4
}

decay_params = {
    "TVSpend":{
        "decay":1,
        "saturation":8
    },
    "Facebook":{
        "decay":13,
        "saturation":7
    },
    "Video":{
        "decay":3,
        "saturation":4
    },
    "Display":{
        "decay":13,
        "saturation":15
    },
    "Search":{
        "decay":3,
        "saturation":4
    },
    "POSRebate":{
        "decay":3,
        "saturation":2
    },
    "TradeDisplayPlacement":{
        "decay":1,
        "saturation":5
    }
}


In [50]:
def objective_function(x,non_marketing,marketing_beta,decay_sat_par_val_dict,decay_params,sign=-1.0):
    #print("Inside Objective Function x=> ",x)
    TVSpend,Facebook,Video,Display,Search = get_marketing_series(x,5)
    df_TVSpend,df_Facebook,df_Video,df_Display,df_Search = get_marketing_df(TVSpend,Facebook,Video,Display,Search)
    tvspend_ads,facebook_ads,video_ads,display_ads,search_ads = get_ads_marketing(df_TVSpend,df_Facebook,df_Video,df_Display,df_Search,decay_sat_par_val_dict,decay_params)
    
    beta_tv = marketing_beta["TVSpend"].values[0]    
    beta_facebook = marketing_beta["Facebook"].values[0]
    beta_video = marketing_beta["Video"].values[0]
    beta_display = marketing_beta["Display"].values[0]
    beta_search = marketing_beta["Search"].values[0]
    constant = non_marketing['non_marketing']
    constant = constant.reset_index()
    constant = constant['non_marketing']
    equation = constant * (tvspend_ads**beta_tv) * (facebook_ads**beta_facebook) * (video_ads**beta_video) * (display_ads**beta_display) * \
                  (search_ads**beta_search)
    equation = np.asarray(list(map(int, equation)))
    
    return (sign * sum(equation))



def objective_functions_derivative(x,non_marketing,marketing_beta,decay_sat_par_val_dict,decay_params,sign=-1.0):
    TVSpend,Facebook,Video,Display,Search = get_marketing_series(x,5)
    df_TVSpend,df_Facebook,df_Video,df_Display,df_Search = get_marketing_df(TVSpend,Facebook,Video,Display,Search)
    tvspend_ads,facebook_ads,video_ads,display_ads,search_ads = get_ads_marketing(df_TVSpend,df_Facebook,df_Video,df_Display,df_Search,decay_sat_par_val_dict,decay_params)
    
    beta_tv = marketing_beta["TVSpend"].values[0]    
    beta_facebook = marketing_beta["Facebook"].values[0]
    beta_video = marketing_beta["Video"].values[0]
    beta_display = marketing_beta["Display"].values[0]
    beta_search = marketing_beta["Search"].values[0]
    constant = non_marketing['non_marketing']
    constant = constant.reset_index()
    constant = constant['non_marketing']
    
    tvspend_der  = constant*(beta_tv*(tvspend_ads**(beta_tv-1))) * (facebook_ads**beta_facebook) * (video_ads**beta_video) * (display_ads**beta_display) * \
                   (search_ads**beta_search)
    tvspend_der  = np.asarray(list(tvspend_der))
            
    facebook_der = constant * (tvspend_ads**beta_tv) * (beta_facebook*(facebook_ads**(beta_facebook-1))) * (video_ads**beta_video) * (display_ads**beta_display) * \
                   (search_ads**beta_search)
    facebook_der = np.asarray(list(facebook_der))
            
    video_der    = constant * (tvspend_ads**beta_tv) * (facebook_ads**beta_facebook) * (beta_video*(video_ads**(beta_video-1))) * (display_ads**beta_display) * \
                   (search_ads**beta_search)
    video_der =    np.asarray(list(video_der))
    
    display_der  = constant * (tvspend_ads**beta_tv) * (facebook_ads**beta_facebook) * (video_ads**beta_video) * \
                   (beta_display*(display_ads**(beta_display-1))) *  (search_ads**beta_search)
    display_der =  np.asarray(list(display_der))       
         
    search_der =   constant * (tvspend_ads**beta_tv) * (facebook_ads**beta_facebook) * (video_ads**beta_video) * (display_ads**beta_display) *\
                   (beta_search*(search_ads**(beta_search-1)))
    search_der =    np.asarray(list(search_der))
    
    dfdx0 = np.column_stack(((sign*tvspend_der),(sign*facebook_der),(sign*video_der),(sign*display_der),(sign*search_der))).ravel()
    return dfdx0


In [51]:
df_weekwise_spending = df_weekwise_spending[df_weekwise_spending["Title"]==title]
max_title_budget = get_title_budget(df_title_budget_constraints,title)
non_marketing = get_Non_Marketing_Week_Values(non_marketing_week_values.copy(),non_marketing_cols,title)
non_marketing_len = len(non_marketing['Week Number'])
print(non_marketing_len)
print("max_title_budget = ",max_title_budget)
non_marketing

53
max_title_budget =  5410182


non_marketing  Week Number
219   11515.940164            0
220    7116.189579            1
221    8440.235768            2
222    5915.855714            3
223    7458.381267            4
224    6238.623360            5
225    6960.635840            6
226    5773.309725            7
227    5752.253106            8
228    5430.094779            9
229    5418.703545           10
230    5558.233235           11
231    5458.966440           12
232    5446.837575           13
233    5438.995020           14
234    4662.831886           15
235    5415.186340           16
236    6086.750935           17
237    6079.574057           18
238    6001.058429           19
239    5929.008839           20
240    7254.519026           21
241    4653.785076           22
242    4767.563967           23
243    4433.462508           24
244    4614.921026           25
245    5725.871423           26
246    5577.688050           27
247    6691.099797           28
248    4449.535637           29
249    5091.426124           30
250    5080.745348           31
251    5783.779078           32
252    5006.231544           33
253    5864.823489           34
254    5181.879238           35
255    4925.719056           36
256    5808.758751           37
257   10667.246200           38
258   11197.258313           39
259   11452.675703           40
260   11192.414895           41
261   11686.943342           42
262    6179.025561           43
263    2961.940776           44
264    2955.727227           45
265    2821.714619           46
266    2943.339205           47
267    2937.164679           48
268    2931.003105           49
269    2924.854457           50
270    3941.434412           51
271    3933.166085           52

In [52]:
### For Inititialisation 

# starting_df = pd.read_excel(path,sheetname = "X0_Lego_Super_DC")
# starting_df = starting_df[starting_df.index<=(non_marketing_len-1)]
# x0 = np.array(starting_df.values).ravel()

x0 = np.array([1,1,1,1,1]*non_marketing_len)
print(len(x0))
x0

265


array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1])

In [53]:
print(objective_functions_derivative(x0,non_marketing,marketing_beta,decay_sat_par_val_dict,decay_params,sign=-1.0))
print(objective_function(x0,non_marketing,marketing_beta,decay_sat_par_val_dict,decay_params,sign=-1.0))
df_title_budget_constraints

[-3.91541966e+01 -2.89050098e+02 -3.46629799e+02 -4.38757320e+02
 -5.64281068e+01 -1.28906913e+01 -9.87101957e+01 -1.43700812e+02
 -1.56390194e+02 -2.33931554e+01 -1.05486073e+01 -8.38550639e+01
 -1.35379739e+02 -1.37774256e+02 -2.20385622e+01 -5.67857988e+00
 -4.68358677e+01 -8.11035516e+01 -7.96648918e+01 -1.32029037e+01
 -5.83105603e+00 -4.98673964e+01 -9.14654601e+01 -8.76919359e+01
 -1.48897261e+01 -4.12258610e+00 -3.65348632e+01 -7.06200333e+01
 -6.63249119e+01 -1.14962845e+01 -3.98854308e+00 -3.66071787e+01
 -7.43749350e+01 -6.85033794e+01 -1.21075476e+01 -2.92291402e+00
 -2.77672748e+01 -5.91736622e+01 -5.34812447e+01 -9.63292175e+00
 -2.61030527e+00 -2.56524154e+01 -5.72154599e+01 -5.07774402e+01
 -9.31414464e+00 -2.23388616e+00 -2.26973410e+01 -5.28573784e+01
 -4.61057818e+01 -8.60468951e+00 -2.03967956e+00 -2.14147245e+01
 -5.19352124e+01 -4.45769092e+01 -8.45456947e+00 -1.92905548e+00
 -2.09167275e+01 -5.26854547e+01 -4.45559415e+01 -8.57670193e+00
 -1.75819725e+00 -1.96779

Title   Budget
0                             Lego Incredibles  8257208
1                     Lego Marvel Super Hero 2  5227928
2                 Lego Ninjago Movie Videogame  6616255
3                                Lego Super DC  7066587
4                                  Lego Worlds  5410182
5                         Cars-3-Driven-to-Win  3142104
6  Scribblenauts Showdown/Scribblenauts Fiesta  1274396

In [54]:
### Constraints Function
def title_budget_constraint(x,budget):
    max_title_spend = budget
    max_title_spend = max_title_spend - sum(x)
    return max_title_spend

def get_start_index(channel):
    s_index = 0
    if(channel=="TVSpend"): s_index=0
    elif(channel=="Facebook"): s_index = 1
    elif(channel=="Video"): s_index=2
    elif(channel=="Display"): s_index =3
    elif(channel == "Search"): s_index=4
    else: s_index=5
    return s_index
 
def channel_constraint(x,constraint_range,df_channel_ranges,channel,title_budget,non_marketing_len):
    s_index = get_start_index(channel)
    channel_sum = 0
    for i in range(0,non_marketing_len):
        channel_sum = channel_sum + x[s_index + (i*5)]
    
    min_channel_range_percentage = df_channel_ranges[df_channel_ranges["Channel"]==channel]["Min_Percentage"].values[0]
    max_channel_range_percentage = df_channel_ranges[df_channel_ranges["Channel"]==channel]["Max_Percentage"].values[0]
    if(constraint_range=="Min_Percentage"):
        return channel_sum - ((min_channel_range_percentage/100)*title_budget)
    else:
        return ((max_channel_range_percentage/100)*title_budget) - channel_sum   
    

In [55]:
### Added this so that we can restrict spending (bounds) for the given weeks.

def isNaN(num):
    return num != num


def bounds_week_spending(title,channel,non_marketing_len,bounds):
    arr = bounds.copy()
    s_index = get_start_index(channel)
    spend_weeks = df_weekwise_spending[df_weekwise_spending["Channel"]==channel]["Week Number"].values[0]
    non_zero_weeks = []
    if(isNaN(spend_weeks)):
        non_zero_weeks = []
    elif(type(spend_weeks)==int):
        non_zero_weeks = [spend_weeks]
    else:
        non_zero_weeks = list(map(int,spend_weeks.split(",")))

    all_weeks = [i for  i in range(0,non_marketing_len)]
    zero_weeks = [x for x in all_weeks if x not in non_zero_weeks]
    for i in range(0,non_marketing_len):
        if(i in zero_weeks):
            l = list(arr[s_index + (i*5)])
            l[1] = 1
            arr[s_index + (i*5)] = tuple(l)
    return arr

bounds = [(1,1000000)]*non_marketing_len*5

for channel in channels:
    print(channel)
    bounds = bounds_week_spending(title,channel,non_marketing_len,bounds)

#bounds = [(1,1000000),(1,1000000),(1,1000000),(1,1000000),(1,1000000)]*non_marketing_len
bounds


TVSpend
Facebook
Video
Display
Search


[(1, 1),
 (1, 1000000),
 (1, 1000000),
 (1, 1000000),
 (1, 1),
 (1, 1),
 (1, 1000000),
 (1, 1000000),
 (1, 1000000),
 (1, 1),
 (1, 1),
 (1, 1000000),
 (1, 1000000),
 (1, 1000000),
 (1, 1),
 (1, 1),
 (1, 1000000),
 (1, 1000000),
 (1, 1000000),
 (1, 1),
 (1, 1000000),
 (1, 1000000),
 (1, 1000000),
 (1, 1000000),
 (1, 1),
 (1, 1000000),
 (1, 1000000),
 (1, 1000000),
 (1, 1000000),
 (1, 1),
 (1, 1000000),
 (1, 1000000),
 (1, 1000000),
 (1, 1000000),
 (1, 1),
 (1, 1000000),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1,

In [56]:
# channel = "Facebook"
# int(actual_week_spending[(actual_week_spending.index==2)][channel].values[0])*1.5

In [57]:
### For restricting week wise spending to 1.5 times actual spending

# def isNaN(num):
#     return num != num


# def bounds_week_spending(title,channel,non_marketing_len,bounds,actual_week_spending):
#     arr = bounds.copy()
#     s_index = get_start_index(channel)
#     spend_weeks = df_weekwise_spending[df_weekwise_spending["Channel"]==channel]["Week Number"].values[0]
#     non_zero_weeks = []
#     if(isNaN(spend_weeks)):
#         non_zero_weeks = []
#     elif(type(spend_weeks)==int):
#         non_zero_weeks = [spend_weeks]
#     else:
#         non_zero_weeks = list(map(int,spend_weeks.split(",")))

#     all_weeks = [i for  i in range(0,non_marketing_len)]
#     zero_weeks = [x for x in all_weeks if x not in non_zero_weeks]
#     for i in range(0,non_marketing_len):
#         if(i in zero_weeks):
#             l = list(arr[s_index + (i*5)])
#             l[1] = 1
#             arr[s_index + (i*5)] = tuple(l)
#         else:
#             l = list(arr[s_index + (i*5)])
#             l[1] = int(1.5*actual_week_spending[(actual_week_spending.index==i)][channel].values[0])
#             arr[s_index + (i*5)] = tuple(l)
#     return arr

# bounds = [(1,1000000)]*non_marketing_len*5

# for channel in channels:
#     print(channel)
#     bounds = bounds_week_spending(title,channel,non_marketing_len,bounds,actual_week_spending)
# bounds

In [58]:
print(bounds[41])
print(channel_constraint(x0,"Max_Percentage",df_channel_ranges,"Search",max_title_budget,non_marketing_len))

(1, 1)
216354.28


In [59]:
def get_min_max_channel_constraints(channels,df_channel_ranges,max_title_budget,non_marketing_len):
    min_channel_constraints = []
    max_channel_constraints = []
    for channel in channels:
        dic_channel_min_constraint = {
            "type":"ineq",
            "fun" :channel_constraint,
            "args":["Min_Percentage",df_channel_ranges,channel,max_title_budget,non_marketing_len]
        }
        dic_channel_max_constraint = {
            "type":"ineq",
            "fun" :channel_constraint,
            "args":["Max_Percentage",df_channel_ranges,channel,max_title_budget,non_marketing_len]
        }
        min_channel_constraints.append(dic_channel_min_constraint)
        max_channel_constraints.append(dic_channel_max_constraint)
    return (min_channel_constraints,max_channel_constraints)

def get_title_budget_constraint(budget):
    dic_title_budget_constraint = {}
    dic_title_budget_constraint.clear()
    dic_title_budget_constraint["type"] = "eq"
    dic_title_budget_constraint["fun"] = title_budget_constraint
    dic_title_budget_constraint["args"] = [budget]
    return dic_title_budget_constraint


In [60]:
title_budget_const = get_title_budget_constraint(max_title_budget.copy())
min_channel_constraints,max_channel_constraints  = get_min_max_channel_constraints(channels,df_channel_ranges,max_title_budget,non_marketing_len)
constraints = []
constraints.append(title_budget_const)
constraints = constraints + min_channel_constraints
constraints = constraints + max_channel_constraints

In [61]:
print(min_channel_constraints)
print(len(constraints))
print(channels)

[{'type': 'ineq', 'fun': <function channel_constraint at 0x00000218A5077488>, 'args': ['Min_Percentage',      Channel  Min_Percentage  Max_Percentage
10   TVSpend              48              75
11  Facebook               0               8
12     Video               8              15
13    Search               0               4
14   Display               0              21, 'TVSpend', 5410182, 53]}, {'type': 'ineq', 'fun': <function channel_constraint at 0x00000218A5077488>, 'args': ['Min_Percentage',      Channel  Min_Percentage  Max_Percentage
10   TVSpend              48              75
11  Facebook               0               8
12     Video               8              15
13    Search               0               4
14   Display               0              21, 'Facebook', 5410182, 53]}, {'type': 'ineq', 'fun': <function channel_constraint at 0x00000218A5077488>, 'args': ['Min_Percentage',      Channel  Min_Percentage  Max_Percentage
10   TVSpend              48              75
11

In [63]:
Nfeval= 0 
def callbackF(Xi):
    global Nfeval
    print("Nfeval = {}  {}   {}   {}".format(Nfeval, Xi[0], Xi[1], Xi[2]))
    Nfeval += 1
    
def get_overallChannelSpend(result,channels,columns,budget):
    TVSpend,Facebook,Video,Display,Search = get_marketing_series(result.x,channels)
    df_result = pd.concat([TVSpend,Facebook,Video,Display,Search],axis=1)
    df_result.columns = columns
    df_1 = df_result.sum(axis=0).to_frame(name=budget).transpose()
    return (df_1,df_result)

import random
def get_starting_point(seed,channels,non_marketing_len,x_prev):
    random.seed(seed)
    xi = []
    for i in range(0,(len(channels)*non_marketing_len)):
        xi.append(random.randint(100000,600000))
    return np.array(xi)

def callbackBasin(X,F,accept):
    global Nfeval
    print("Nfeval = {}  , F = {}  , accept =  {} ".format(Nfeval,F,accept))
    Nfeval += 1

# Running Optimiser

In [52]:
from datetime import datetime
s_time = datetime.now().timestamp()
result = minimize(objective_function,x0,args = (non_marketing,marketing_beta,decay_sat_par_val_dict,decay_params,-1),method = "SLSQP" ,bounds = bounds, constraints = constraints,jac = objective_functions_derivative,callback = callbackF,options = {"maxiter":40,"disp":True},tol=.001)
e_time = datetime.now().timestamp()

print("s_time = ",s_time)
print("e_time = ",e_time)
print("sec = ",(e_time-s_time))

Nfeval = 0  285.02405532995994   325.6595273618836   466.47778819963844
Nfeval = 1  344.21044087343233   291.74213783522544   512.9842399911585
Nfeval = 2  350.52907921327653   291.17694985696346   515.8597575856052
Nfeval = 3  370.584048667337   289.4533708767088   524.3218562948292
Nfeval = 4  420.9768474015641   286.63756474999695   545.4769670482865
Nfeval = 5  539.0547897617536   280.63964636290814   594.6144746650283
Nfeval = 6  689.5182078766754   275.25271861787206   657.4548428331037
Nfeval = 7  963.6327697096679   263.7162933713203   767.9045823835806
Nfeval = 8  1644.498319370939   232.62339647215035   1035.5926788634797
Nfeval = 9  2805.594989444285   183.9911375826008   1490.4781252431267
Nfeval = 10  4510.809091885934   119.19467229215797   2156.2997615064323
Nfeval = 11  6799.9622851116   44.16060962170694   3046.3064087411303
Nfeval = 12  7382.455256749012   48.81309831147459   3265.7614156851178
Nfeval = 13  10252.725243154335   63.81229619274608   4349.388959902964
Nf

In [82]:
### For Single Iteration with given budget
from datetime import datetime

Nfeval = 0
budget =2569724
title_budget_const = get_title_budget_constraint(budget)
min_channel_constraints,max_channel_constraints  = get_min_max_channel_constraints(channels,df_channel_ranges,budget,non_marketing_len)
constraints = []
constraints.append(title_budget_const)
constraints = constraints + min_channel_constraints
constraints = constraints + max_channel_constraints

s_time = datetime.now().timestamp()
result = minimize(objective_function,x0,args = (non_marketing,marketing_beta,decay_sat_par_val_dict,decay_params,-1),method = "SLSQP" ,bounds = bounds, constraints = constraints,jac = objective_functions_derivative,callback = callbackF,options = {"maxiter":40,"disp":True},tol=.001)
e_time = datetime.now().timestamp()

print("s_time = ",s_time)
print("e_time = ",e_time)
print("sec = ",(e_time-s_time))

Nfeval = 0  1.0   3930.2111300011575   35698.149699053945
Nfeval = 1  1.0   3915.9237353431345   35679.18215737462
Nfeval = 2  1.0   3916.0842129002763   35679.188349547774
Optimization terminated successfully.    (Exit mode 0)
            Current function value: -860410.0
            Iterations: 3
            Function evaluations: 5
            Gradient evaluations: 3
s_time =  1566551116.525009
e_time =  1566551137.661273
sec =  21.136264085769653


In [59]:
columns = ["TVSpend","Facebook","Video","Display","Search"]
df_1,df_result = get_overallChannelSpend(result,5,columns,budget)
print("Total Spend = ",df_result.values.sum())
print(df_result.sum(axis=0).values)
df_result

Total Spend =  2047876.9999992943
[1.37207759e+06 2.33257234e+05 3.19648556e+05 2.10000000e+01
 1.22872620e+05]


TVSpend      Facebook         Video  Display        Search
0   302338.518597  64900.083159  65040.905745      1.0  11265.237658
1   302281.606335  64490.613950  64626.191771      1.0  11197.715861
2   302267.465068  64389.536426  64510.203593      1.0  11178.831225
3   242092.000000   8113.000000  11902.000000      1.0  11151.624892
4    34553.000000      1.000000  17727.000000      1.0  11144.339538
5    85532.000000      1.000000  64299.255357      1.0  11144.485630
6    91002.000000      1.000000  31529.000000      1.0  11155.764139
7    11998.000000  15334.000000      1.000000      1.0  11157.257480
8        1.000000  16015.000000      1.000000      1.0  11157.350103
9        1.000000      1.000000      1.000000      1.0  11155.409348
10       1.000000      1.000000      1.000000      1.0  11154.604127
11       1.000000      1.000000      1.000000      1.0      1.000000
12       1.000000      1.000000      1.000000      1.0      1.000000
13       1.000000      1.000000      1.000000      1.0      1.000000
14       1.000000      1.000000      1.000000      1.0      1.000000
15       1.000000      1.000000      1.000000      1.0      1.000000
16       1.000000      1.000000      1.000000      1.0      1.000000
17       1.000000      1.000000      1.000000      1.0      1.000000
18       1.000000      1.000000      1.000000      1.0      1.000000
19       1.000000      1.000000      1.000000      1.0      1.000000
20       1.000000      1.000000      1.000000      1.0      1.000000

### For Running Multiple times to get the best result

In [208]:
noOfIterations = 15
results = []
x_in = x0
Nfeval= 0 
s_time = datetime.now().timestamp()
for i in range(0,noOfIterations):
    print("Iteration is ===========>>>>",i)
    result = minimize(objective_function,x_in,args = (non_marketing,marketing_beta,decay_sat_par_val_dict,decay_params,-1),method = "SLSQP" ,bounds = bounds, constraints = constraints,jac = objective_functions_derivative,callback = callbackF,options = {"maxiter":40,"disp":True},tol=.001)
    print("result for iteration {} = {}".format(i,result.fun*-1))
    results.append(result)
    x_in = get_starting_point(result.fun*-1,channels,non_marketing_len,result.x)
    Nfeval= 0 
e_time = datetime.now().timestamp()

print("sec is =>>",(e_time-s_time))


Iteration is ===========>>>> 0
Nfeval = 0  247878.25813484343   256607.32510699733   455805.5544982495
Nfeval = 1  247878.59145110706   256607.35398718438   455805.47575449804
Optimization terminated successfully.    (Exit mode 0)
            Current function value: -630177.0
            Iterations: 2
            Function evaluations: 3
            Gradient evaluations: 2
result for iteration 0 = 630177.0
Iteration is ===========>>>> 1
Nfeval = 0  97817.17361810291   186200.1774965365   274673.3780292907
Nfeval = 1  97759.7817899177   186142.78962884826   274599.2218077292
Nfeval = 2  97757.8357080327   186140.84795065975   274596.02927150467
Nfeval = 3  97757.89900554251   186140.91604323743   274595.5875148791
Nfeval = 4  97758.0696587624   186141.10459459285   274594.29368399037
Nfeval = 5  97758.03833768386   186141.10300867038   274592.346538747
Nfeval = 6  97757.83094927183   186140.96382797734   274587.8729735045
Nfeval = 7  97757.32244523193   186140.57257983665   274580.344877

In [179]:
max_result = results[0].fun*-1
result_selected = results[0]
for i in range(1,noOfIterations):
    if((results[i].fun*-1) >=max_result):
        max_result = results[i].fun*-1
        result_selected = results[i]

result_selected  

     fun: -630339.0
     jac: array([-9.62122230e-04, -6.86110975e-03, -4.63208543e-03, -2.67248123e+03,
       -2.50613415e-03, -4.34891959e+02, -5.05651910e-02, -4.20559424e-04,
       -4.50747219e-03, -3.19466062e-03, -1.14317560e+03, -2.13740556e-03,
       -3.41712630e+02, -5.45663319e-02, -2.32791891e-04, -3.07657590e-03,
       -2.10376486e-03, -6.86179203e+02, -1.92492532e-03, -2.85211416e+02,
       -3.64665008e-02, -1.30069577e-04, -2.07209899e-03, -1.92172900e-03,
       -3.98219984e+02, -1.40665213e-03, -4.26572851e-03, -2.74577400e-02,
       -8.01380668e-05, -1.76134715e-03, -1.90468892e-03, -2.68281240e+02,
       -1.09765361e-03, -3.34605152e-03, -2.21504440e-02, -7.67109991e-05,
       -1.90507098e-03, -2.38790355e-03, -2.38765030e+02, -1.08172424e-03,
       -9.53205350e-04, -2.22764656e-02, -1.29540688e-04, -3.62941752e-03,
       -5.24666054e-03, -3.84178056e+02, -1.87627104e-03, -1.41842598e-03,
       -4.02638720e-02, -1.21428102e-04, -3.41933107e-03, -6.69256817e

In [240]:
TVSpend,Facebook,Video,Display,Search = get_marketing_series(result.x,5)
df_result = pd.concat([TVSpend,Facebook,Video,Display,Search],axis=1)
df_result.columns = ["TVSpend","Facebook","Video","Display","Search"]
df_result

TVSpend       Facebook          Video  Display         Search
0   496289.599739  269195.652877  332179.883437      1.0  117404.505701
1   772200.000588  134463.897799  231610.444225      1.0   59368.715562
2   484152.024085  110203.237597  241057.654897      1.0   76382.211402
3   458238.192064   64106.830587  169437.943206      1.0   65881.285346
4   127995.428638       1.000000   99117.127967      1.0   35935.086790
5    57003.990040       1.000000       1.000000      1.0   57855.614933
6        1.000000       1.000000       1.000000      1.0       1.000000
7        1.000000       1.000000       1.000000      1.0       1.000000
8        1.000000       1.000000       1.000000      1.0       1.000000
9        1.000000       1.000000       1.000000      1.0       1.000000
10       1.000000       1.000000       1.000000      1.0       1.000000
11       1.000000       1.000000       1.000000      1.0       1.000000
12       1.000000       1.000000       1.000000      1.0       1.000000
13       1.000000       1.000000       1.000000      1.0       1.000000
14       1.000000       1.000000       1.000000      1.0       1.000000
15       1.000000       1.000000       1.000000      1.0       1.000000
16       1.000000       1.000000       1.000000      1.0       1.000000
17       1.000000       1.000000       1.000000      1.0       1.000000
18       1.000000       1.000000       1.000000      1.0       1.000000
19       1.000000       1.000000       1.000000      1.0       1.000000
20       1.000000       1.000000       1.000000      1.0       1.000000
21       1.000000       1.000000       1.000000      1.0       1.000000
22       1.000000       1.000000       1.000000      1.0       1.000000
23       1.000000       1.000000       1.000000      1.0       1.000000
24       1.000000       1.000000       1.000000      1.0       1.000000
25       1.000000       1.000000       1.000000      1.0       1.000000
26       1.000000       1.000000       1.000000      1.0       1.000000
27       1.000000       1.000000       1.000000      1.0       1.000000
28       1.000000       1.000000       1.000000      1.0       1.000000
29       1.000000       1.000000       1.000000      1.0       1.000000
30       1.000000       1.000000       1.000000      1.0       1.000000
31       1.000000       1.000000       1.000000      1.0       1.000000
32       1.000000       1.000000       1.000000      1.0       1.000000
33       1.000000       1.000000       1.000000      1.0       1.000000
34       1.000000       1.000000       1.000000      1.0       1.000000
35       1.000000       1.000000       1.000000      1.0       1.000000
36       1.000000       1.000000       1.000000      1.0       1.000000
37       1.000000       1.000000       1.000000      1.0       1.000000
38       1.000000       1.000000       1.000000      1.0       1.000000

In [64]:
### For Basin Hopping to get global optima

from datetime import datetime
Nfeval= 0 
budget = 2838289
title_budget_const = get_title_budget_constraint(budget)
min_channel_constraints,max_channel_constraints  = get_min_max_channel_constraints(channels,df_channel_ranges,budget,non_marketing_len)
constraints = []
constraints.append(title_budget_const)
constraints = constraints + min_channel_constraints
constraints = constraints + max_channel_constraints

minimizer_kwargs = {"method": "SLSQP","constraints":constraints,"bounds":bounds,"args":(non_marketing,marketing_beta,decay_sat_par_val_dict,decay_params,-1),"jac":objective_functions_derivative}

basin_result = basinhopping(objective_function, x0, minimizer_kwargs=minimizer_kwargs,niter=5,
            disp=True,stepsize=.001,niter_success=5,seed = 7,callback = callbackBasin)

basinhopping step 0: f -882571
basinhopping step 1: f -887401 trial_f -887401 accepted 1  lowest_f -887401
found new global minimum on step 1 with function value -887401
Nfeval = 0  , F = -887401.0  , accept =  True 
basinhopping step 2: f -887402 trial_f -887402 accepted 1  lowest_f -887402
found new global minimum on step 2 with function value -887402
Nfeval = 1  , F = -887402.0  , accept =  True 
basinhopping step 3: f -887402 trial_f -887401 accepted 0  lowest_f -887402
Nfeval = 2  , F = -887401.0  , accept =  False 
basinhopping step 4: f -887402 trial_f -887401 accepted 0  lowest_f -887402
Nfeval = 3  , F = -887401.0  , accept =  False 
basinhopping step 5: f -887402 trial_f -887401 accepted 0  lowest_f -887402
Nfeval = 4  , F = -887401.0  , accept =  False 


In [65]:
basin_result.lowest_optimization_result

     fun: -887402.0
     jac: array([-1.11656659e+02, -9.71962857e-03, -1.45303334e-02, -1.64836572e-02,
       -1.60916950e+02, -3.55191636e+01, -5.09152802e-03, -8.57368193e-03,
       -7.59589028e-03, -6.44577774e+01, -2.86429917e+01, -5.32072138e-03,
       -9.45376000e-03, -7.30659940e-03, -5.98420566e+01, -1.51625284e+01,
       -3.76736767e-03, -6.69812298e-03, -4.71957165e-03, -3.52534273e+01,
       -4.94790555e-04, -4.93457926e-03, -8.26718563e-03, -5.73183728e-03,
       -4.08076456e+01, -2.08595000e-04, -4.22909821e-03, -7.05569639e-03,
       -4.35330550e-03, -3.13212324e+01, -1.56025255e-04, -4.86957147e-03,
       -7.95494790e-03, -4.45723924e-03, -3.28287058e+01, -9.58048722e-05,
       -4.31661872e-03, -8.31112338e-03, -4.25946262e-03, -2.55692482e+01,
       -9.43881450e-05, -4.59532884e-03, -1.06741694e-02, -4.92228844e-03,
       -2.41837338e+01, -8.80922767e-05, -4.63339622e-03, -1.31415346e-02,
       -5.39219841e-03, -2.18677062e+01, -8.68949726e-05, -4.93754741e

In [66]:
columns = ["TVSpend","Facebook","Video","Display","Search"]
df_2,df_basin_result = get_overallChannelSpend(basin_result.lowest_optimization_result,5,columns,budget)
df_2

TVSpend       Facebook          Video        Display  Search
2838289  1.589389e+06  227063.117795  425743.345643  596040.692198    53.0

In [67]:
df_basin_result

TVSpend      Facebook         Video       Display  Search
0        1.000000  84806.622177  68029.398205  75906.176073     1.0
1        1.000000  22836.939834  27555.817894  40317.527822     1.0
2        1.000000  17635.436803  25459.104342  37717.807583     1.0
3        1.000000   5818.018432  15189.050009  30596.829982     1.0
4   159536.126285   6991.798299  17517.267555  31706.625219     1.0
5   159028.885498   2648.247963  13522.833086  29034.696566     1.0
6   158936.452939   2469.137625  14146.108598  29161.262890     1.0
7   158826.664769      1.000000      1.000000      1.000000     1.0
8        1.000000      1.000000      1.000000      1.000000     1.0
9        1.000000      1.000000      1.000000      1.000000     1.0
10       1.000000      1.000000      1.000000      1.000000     1.0
11       1.000000      1.000000      1.000000      1.000000     1.0
12       1.000000      1.000000      1.000000      1.000000     1.0
13       1.000000      1.000000      1.000000      1.000000     1.0
14       1.000000      1.000000      1.000000      1.000000     1.0
15       1.000000      1.000000      1.000000      1.000000     1.0
16       1.000000      1.000000      1.000000      1.000000     1.0
17       1.000000      1.000000      1.000000      1.000000     1.0
18       1.000000      1.000000      1.000000      1.000000     1.0
19       1.000000      1.000000      1.000000      1.000000     1.0
20       1.000000      1.000000      1.000000      1.000000     1.0
21       1.000000      1.000000      1.000000      1.000000     1.0
22       1.000000      1.000000      1.000000      1.000000     1.0
23       1.000000      1.000000      1.000000      1.000000     1.0
24       1.000000      1.000000      1.000000      1.000000     1.0
25       1.000000      1.000000      1.000000      1.000000     1.0
26       1.000000      1.000000  33660.558123      1.000000     1.0
27       1.000000      1.000000  18898.924946      1.000000     1.0
28       1.000000      1.000000      1.000000      1.000000     1.0
29       1.000000      1.000000      1.000000      1.000000     1.0
30       1.000000      1.000000      1.000000      1.000000     1.0
31       1.000000      1.000000      1.000000      1.000000     1.0
32       1.000000      1.000000      1.000000      1.000000     1.0
33       1.000000  17332.330097  24440.004861  45464.563345     1.0
34       1.000000  11988.990037  19689.432940  37465.575321     1.0
35       1.000000   5363.126971  14150.289369  31729.350008     1.0
36       1.000000      1.000000      1.000000      1.000000     1.0
37  158805.355370   4882.916523  16316.741234  31793.667248     1.0
38  158886.429037  12265.732774  25742.543031  37428.251532     1.0
39  158862.791787  10466.071042  24486.275857  36087.200034     1.0
40  158840.717298   8745.746996  23356.117027  34903.417832     1.0
41  158816.053701   6726.848825  21712.961748  33505.999728     1.0
42  158806.362287   6048.153396  21834.916817  33184.741016     1.0
43       1.000000      1.000000      1.000000      1.000000     1.0
44       1.000000      1.000000      1.000000      1.000000     1.0
45       1.000000      1.000000      1.000000      1.000000     1.0
46       1.000000      1.000000      1.000000      1.000000     1.0
47       1.000000      1.000000      1.000000      1.000000     1.0
48       1.000000      1.000000      1.000000      1.000000     1.0
49       1.000000      1.000000      1.000000      1.000000     1.0
50       1.000000      1.000000      1.000000      1.000000     1.0
51       1.000000      1.000000      1.000000      1.000000     1.0
52       1.000000      1.000000      1.000000      1.000000     1.0

In [31]:
TVSpend,Facebook,Video,Display,Search,POSRebate,TradeDisplayPlacement = get_marketing_series(basin_result.x)
df_result = pd.concat([TVSpend,Facebook,Video,Display,Search,POSRebate,TradeDisplayPlacement],axis=1)
df_result.columns = ["TVSpend","Facebook","Video","Display","Search","POSRebate","TradeDisplayPlacement"]
df_result

TVSpend       Facebook          Video        Display         Search  \
0   243671.828594  330951.254515  457345.923204  532601.436939  112949.556665   
1   192994.318179   26890.017543  178460.828772  179141.829090   14981.984444   
2   191500.395876   76507.401929  205694.947231       1.000000       1.000000   
3   184255.555150   28689.660473       1.000000       1.000000       1.000000   
4   179968.150595       1.000000       1.000000       1.000000       1.000000   
5        1.000000       1.000000       1.000000       1.000000       1.000000   
6        1.000000       1.000000       1.000000       1.000000       1.000000   
7        1.000000       1.000000       1.000000       1.000000       1.000000   
8        1.000000       1.000000       1.000000       1.000000       1.000000   
9        1.000000       1.000000       1.000000       1.000000       1.000000   
10       1.000000       1.000000       1.000000       1.000000       1.000000   
11       1.000000       1.000000       1.000000       1.000000       1.000000   
12       1.000000       1.000000       1.000000       1.000000       1.000000   
13       1.000000       1.000000       1.000000       1.000000       1.000000   
14       1.000000       1.000000       1.000000       1.000000       1.000000   
15       1.000000       1.000000       1.000000       1.000000       1.000000   
16       1.000000       1.000000       1.000000       1.000000       1.000000   
17       1.000000       1.000000       1.000000       1.000000       1.000000   
18       1.000000       1.000000       1.000000       1.000000       1.000000   
19       1.000000       1.000000       1.000000       1.000000       1.000000   
20       1.000000       1.000000       1.000000       1.000000       1.000000   
21       1.000000       1.000000       1.000000       1.000000       1.000000   
22       1.000000       1.000000       1.000000       1.000000       1.000000   
23       1.000000       1.000000       1.000000       1.000000       1.000000   
24       1.000000       1.000000       1.000000       1.000000       1.000000   
25       1.000000       1.000000       1.000000       1.000000       1.000000   
26       1.000000       1.000000       1.000000       1.000000       1.000000   
27       1.000000       1.000000       1.000000       1.000000       1.000000   
28       1.000000       1.000000       1.000000       1.000000       1.000000   
29       1.000000       1.000000       1.000000       1.000000       1.000000   
30       1.000000       1.000000       1.000000       1.000000       1.000000   
31       1.000000       1.000000       1.000000       1.000000       1.000000   
32       1.000000       1.000000       1.000000       1.000000       1.000000   
33       1.000000       1.000000       1.000000       1.000000       1.000000   
34       1.000000       1.000000       1.000000       1.000000       1.000000   
35       1.000000       1.000000       1.000000       1.000000       1.000000   
36       1.000000       1.000000       1.000000       1.000000       1.000000   
37       1.000000       1.000000       1.000000       1.000000       1.000000   
38       1.000000       1.000000       1.000000       1.000000       1.000000   
39       1.000000       1.000000       1.000000       1.000000       1.000000   
40       1.000000       1.000000       1.000000       1.000000       1.000000   
41       1.000000       1.000000       1.000000       1.000000       1.000000   
42       1.000000       1.000000       1.000000       1.000000       1.000000   
43       1.000000       1.000000       1.000000       1.000000       1.000000   
44       1.000000       1.000000       1.000000       1.000000       1.000000   
45       1.000000       1.000000       1.000000       1.000000       1.000000   
46       1.000000       1.000000       1.000000       1.000000       1.000000   
47       1.000000       1.000000       1.000000       1.000000       1.000000   
48       1.000000       1.000000       1.

In [19]:
## For Running Multiple basin hopping with different budget step size
budget_units = {}
budget_results = {}
budget_week_df = {}
ranges = [(500000,4000000,500000)]
budget_channelSpend = pd.DataFrame()
Nfeval= 0 
for rangei in ranges:
    print("--Rangei is ===>>>",rangei)
    for budget in range(rangei[0],rangei[1],rangei[2]):
        print("---------Budget is------------ >>>>",budget)
        title_budget_const = get_title_budget_constraint(budget)
        min_channel_constraints,max_channel_constraints  = get_min_max_channel_constraints(channels,df_channel_ranges,budget,non_marketing_len)
        constraints = []
        constraints.append(title_budget_const)
        constraints = constraints + min_channel_constraints
        constraints = constraints + max_channel_constraints
        
        minimizer_kwargs = {"method": "SLSQP","constraints":constraints,"bounds":bounds,"args":(non_marketing,marketing_beta,decay_sat_par_val_dict,decay_params,-1),"jac":objective_functions_derivative}

        basin_result = basinhopping(objective_function, x0, minimizer_kwargs=minimizer_kwargs,niter=5,
            disp=True,stepsize=0.001,niter_success=5,seed = 7,callback = callbackBasin)
        result_budget = basin_result.lowest_optimization_result
        budget_units[budget] = (result_budget.fun * -1)
        budget_results[budget] = result_budget
        columns = ["TVSpend","Facebook","Video","Display","Search"]
        df_1,df_result = get_overallChannelSpend(result_budget,5,columns,budget)
        budget_week_df[budget] = df_result
        print("Total Spend = ",df_result.values.sum())
        budget_channelSpend = budget_channelSpend.append(df_1)
        Nfeval = 0

--Rangei is ===>>> (500000, 4000000, 500000)
---------Budget is------------ >>>> 500000
basinhopping step 0: f -499965
basinhopping step 1: f -502789 trial_f -502789 accepted 1  lowest_f -502789
found new global minimum on step 1 with function value -502789
Nfeval = 0  , F = -502789.0  , accept =  True 
basinhopping step 2: f -503522 trial_f -503522 accepted 1  lowest_f -503522
found new global minimum on step 2 with function value -503522
Nfeval = 1  , F = -503522.0  , accept =  True 
basinhopping step 3: f -503522 trial_f -502767 accepted 0  lowest_f -503522
Nfeval = 2  , F = -502767.0  , accept =  False 
basinhopping step 4: f -503522 trial_f -503390 accepted 0  lowest_f -503522
Nfeval = 3  , F = -503390.0  , accept =  False 
basinhopping step 5: f -503522 trial_f -503522 accepted 1  lowest_f -503522
Nfeval = 4  , F = -503522.0  , accept =  True 
Total Spend =  500000.0003175595
---------Budget is------------ >>>> 1000000
basinhopping step 0: f -523140
basinhopping step 1: f -525230

In [21]:
budget_units
budget_df = pd.DataFrame.from_dict(budget_units,orient='index')
budget_df.columns = ['Units']
budget_df

Units
500000   503522.0
1000000  528531.0
2500000  557111.0
3500000  569896.0
2000000  549975.0
3000000  565512.0
1500000  541259.0

In [22]:
budget_channelSpend

TVSpend       Facebook          Video  Display         Search
500000   3.350000e+05   59877.489254   86361.227594     21.0   18740.283474
1000000  6.700000e+05   96578.636380  192743.159681     21.0   40657.204078
1500000  1.005007e+06  179997.289993  256786.172863     21.0   58188.176693
2000000  1.340000e+06  239999.928806  344283.591643     21.0   75695.481065
2500000  1.675000e+06  299605.232493  437245.722512     21.0   88128.049427
3000000  2.010000e+06  355585.387758  513593.968443     21.0  120799.643786
3500000  2.345000e+06  378104.052794  620204.285382     21.0  156670.661827

In [29]:
budget_week_df[3500000]

TVSpend       Facebook          Video  Display        Search
0   306825.257409  212651.737919  256555.246237      1.0  56482.682362
1   295750.442797   73128.230943  129432.009949      1.0      1.997219
2   293016.373688   35953.540836   93529.265175      1.0   1464.191768
3   290306.404587    1759.495464   35531.764312      1.0     18.176974
4   289580.813223       1.000000   20988.284170      1.0    199.977248
5   289474.511501       1.000000   24614.829309      1.0   5881.782282
6   290051.616839       1.000000   59538.886230      1.0   4430.950861
7   289981.579958   30710.446270       1.000000      1.0  17707.865134
8        1.000000   23885.601363       1.000000      1.0  23910.174709
9        1.000000       1.000000       1.000000      1.0  22911.216241
10       1.000000       1.000000       1.000000      1.0  23651.647030
11       1.000000       1.000000       1.000000      1.0      1.000000
12       1.000000       1.000000       1.000000      1.0      1.000000
13       1.000000       1.000000       1.000000      1.0      1.000000
14       1.000000       1.000000       1.000000      1.0      1.000000
15       1.000000       1.000000       1.000000      1.0      1.000000
16       1.000000       1.000000       1.000000      1.0      1.000000
17       1.000000       1.000000       1.000000      1.0      1.000000
18       1.000000       1.000000       1.000000      1.0      1.000000
19       1.000000       1.000000       1.000000      1.0      1.000000
20       1.000000       1.000000       1.000000      1.0      1.000000

In [ ]:
TVSpend,Facebook,Video,Display,Search,POSRebate,TradeDisplayPlacement = get_marketing_series(basin_result.x)
df_result = pd.concat([TVSpend,Facebook,Video,Display,Search,POSRebate,TradeDisplayPlacement],axis=1)
df_result.columns = ["TVSpend","Facebook","Video","Display","Search","POSRebate","TradeDisplayPlacement"]
df_result

### For Multiple Iterations with different step sized of Budget

In [83]:
budget_units = {}
ranges = [(1000000,4000000,500000)]
budget_channelSpend = pd.DataFrame()
for rangei in ranges:
    print("--Rangei is ===>>>",rangei)
    for budget in range(rangei[0],rangei[1],rangei[2]):
        print("---------Budget is------------ >>>>",budget)
        title_budget_const = get_title_budget_constraint(budget)
        min_channel_constraints,max_channel_constraints  = get_min_max_channel_constraints(channels,df_channel_ranges,budget,non_marketing_len)
        constraints = []
        constraints.append(title_budget_const)
        constraints = constraints + min_channel_constraints
        constraints = constraints + max_channel_constraints
        Nfeval= 0 
        result_budget = minimize(objective_function,x0,args = (non_marketing,marketing_beta,decay_sat_par_val_dict,decay_params,-1),method = "SLSQP" ,bounds = bounds, constraints = constraints,jac = objective_functions_derivative,callback = callbackF,options = {"maxiter":60,"disp":True},tol=.001)
        print(result_budget.fun)
        budget_units[budget] = (result_budget.fun * -1)
        columns = ["TVSpend","Facebook","Video","Display","Search"]
        df_1,df_result = get_overallChannelSpend(result_budget,5,columns)
        print("Total Spend = ",df_result.values.sum())
        budget_channelSpend = budget_channelSpend.append(df_1)
        Nfeval = 0

--Rangei is ===>>> (125000, 1000000, 125000)
---------Budget is------------ >>>> 125000
Nfeval = 0  10225.22822561117   2961.588204410661   3394.953920121915
Nfeval = 1  10225.239856453569   2961.8319311470354   3395.2612320393055
Optimization terminated successfully.    (Exit mode 0)
            Current function value: -461493.0
            Iterations: 2
            Function evaluations: 3
            Gradient evaluations: 2
-461493.0
Total Spend =  125000.00000137466
---------Budget is------------ >>>> 250000
Nfeval = 0  20381.532927937194   5461.595701596856   6252.10624804074
Nfeval = 1  20381.48447162863   5461.728276166511   6252.277857756986
Optimization terminated successfully.    (Exit mode 0)
            Current function value: -480710.0
            Iterations: 2
            Function evaluations: 3
            Gradient evaluations: 2
-480710.0
Total Spend =  249999.99999977055
---------Budget is------------ >>>> 375000
Nfeval = 0  30537.625255056475   7961.573777593676   9109

In [84]:
budget_units

{125000: 461493.0,
 250000: 480710.0,
 375000: 491832.0,
 500000: 501508.0,
 625000: 511504.0,
 750000: 519379.0,
 875000: 524142.0}

In [85]:
budget_df = pd.DataFrame.from_dict(budget_units,orient='index')
budget_df.columns = ['Units']
budget_df

Units
250000  480710.0
375000  491832.0
750000  519379.0
875000  524142.0
125000  461493.0
625000  511504.0
500000  501508.0

In [86]:
budget_channelSpend 

TVSpend       Facebook          Video  Display        Search
125000   81250.000001   15000.000000   19979.296806     21.0   8749.703194
250000  162500.000000   30000.000000   39979.223464     21.0  17499.776536
375000  243749.999999   45000.000000   59979.105216     21.0  26249.894783
500000  325000.000001   60000.000000   79979.045362     21.0  34999.954638
625000  406251.671411   74999.791224   99978.835675     21.0  43748.701690
750000  487499.999997   89999.999999  119978.999999     21.0  52499.999999
875000  568751.999836  104999.703529  139978.760395     21.0  61248.536230